In [61]:
import numpy as np
import math

wall_dict = {
            "Legend": ['N','E','S','W'],
            "Unknown": ['?','?','?','?'],
            0 : [0,0,0,0],
            1 : [1,0,0,0],
            2 : [0,1,0,0],
            4 : [0,0,1,0],
            8 : [0,0,0,1],
            3 : [1,1,0,0],
            5 : [1,0,1,0],
            9 : [1,0,0,1],
            6 : [0,1,1,0],
            10 : [0,1,0,1],
            12 : [0,0,1,1],
            7 : [1,1,1,0],
            11 : [1,1,0,1],
            13 : [1,0,1,1],
            14 : [0,1,1,1],
            15 : [1,1,1,1],

        }
# Takes in an average IMU readings and forces it to be a 
# cardinal direction
def angle_to_heading(theta):
    belief_heading = 0
    sd = 25
    if (90 - sd) <= theta <= (90+sd):
        belief_heading = 'North'
    elif (180-sd) <= theta <= (180 +sd):
        belief_heading = 'West'
    elif (270-sd) <= theta <= (270+sd):
        belief_heading = 'South'
    elif theta >= (360-sd) or theta <= (0+sd):
        belief_heading = 'East'
        
    return belief_heading

# Given current robot heading and possible_heading, calculates the change in
# roation needed to face the new heading 
def turnNeeded(heading, possible_heading):
    
    if heading == "North":
        if possible_heading == "North":
            rotate = 0 
        elif possible_heading == "East":
            rotate = 90
        elif possible_heading == "South":
            rotate = 180
        elif possible_heading == "West":
            rotate = -90
    
    elif heading == "East":
        if possible_heading == "North":
            rotate = -90 
        elif possible_heading == "East":
            rotate = 0
        elif possible_heading == "South":
            rotate = 90
        elif possible_heading == "West":
            rotate = 180
            
    elif heading == "South":
        if possible_heading == "North":
            rotate = 180 
        elif possible_heading == "East":
            rotate = -90
        elif possible_heading == "South":
            rotate = 0
        elif possible_heading == "West":
            rotate = 90
    
    elif heading == "West":
        if possible_heading == "North":
            rotate = 90 
        elif possible_heading == "East":
            rotate = 180
        elif possible_heading == "South":
            rotate = -90
        elif possible_heading == "West":
            rotate = 0
    
    return rotate

# Takes a cell's row and column index and returns original indexer
def get_index_from_rc(cell_r,cell_c,size=16):
    return (size*cell_r) + cell_c

def get_index_from_xy(x,y):
    cell_c = math.floor(x/180)
    cell_r = math.floor((2880-y)/180)
    return get_index_from_rc(cell_r,cell_c)

# Takes a cell's index and returns row and column index
def get_rc_from_index(index,size=16):
    return divmod(index,size)

# Takes a cell's index and returns x & y cordinate in mm 
def get_xy_from_index(index,size=16):
    cell_r,cell_c = get_rc_from_index(index,size)
    return get_xy_from_rc(cell_r,cell_c,size)

# Takes a cell's row and column index and returns x & y cordinate in mm 
def get_xy_from_rc(cell_r,cell_c,size=16):
    center_x = ((90) + (c*180))
    center_y = (((180*size) - 90) - (r*180))
    return center_x,center_y

# Takes the robot sensor readings and heading and converts it to a wall configuration (default assumes robot heading is North)
def sensor_readings_to_wall_config(sr,heading='North'):
    wall_config = wall_dict['Legend']
    for i in range(len(sr)):
        if heading == 'North':
            if sr[i] <= .09:
                wall_config[i] = 1
            else:
                wall_config[i] = 0
        elif heading == 'East':
            if sr[i] <= .09:
                wall_config[(i+1)%4] = 1
            else:
                wall_config[(i+1)%4] = 0
        elif heading == 'South':
            if sr[i] <= .09:
                wall_config[(i+2)%4] = 1
            else:
                wall_config[(i+2)%4] = 0
        elif heading == 'West':
            if sr[i] <= .09:
                wall_config[(i+3)%4] = 1
            else:
                wall_config[(i+3)%4] = 0
    return wall_config
class wall:
    # TODO: create a non-exsistant wall
    def __init__(self, cell_index, cell_center_x, cell_center_y, wall_index, wall_valid):
        self.translation_x = cell_center_x
        self.translation_y = cell_center_y
        self.translation_z = 25
        if wall_valid == 1:
            self.valid = True
        else:
            self.valid = False
        if wall_index == 0:
            self.direction = 'North'
            self.translation_y += 90
        elif wall_index == 1:
            self.direction = 'East'
            self.translation_x += 90
        elif wall_index == 2:
            self.direction = 'South'
            self.translation_y -= 90
        elif wall_index == 3:
            self.direction = 'West'
            self.translation_x -= 90

        
class cell:
    def __init__(self, index = 0, wall_code = 'Unknown', discovered = False, size = 16):
        
        self.index = index
        self.wall_code = wall_code
        self.wall_config = wall_dict[wall_code]
        self.discovered = discovered
        
        # Sets the row and column index relative to a size x size maze (default 16x16)
        self.cell_row, self.cell_col = divmod(index,size)
        
        # Sets the row and column index relative of adjacent cells
        
        # Cell to the North
        self.cell_to_north_row   = self.cell_row - 1 if self.cell_row - 1 >= 0 else 0
        self.cell_to_north_col   = self.cell_col
        self.cell_index_to_north = get_index_from_rc(self.cell_to_north_row,self.cell_to_north_col)
        
        # Cell to the South
        self.cell_to_south_row = self.cell_row + 1 if self.cell_row + 1 <= 15 else 15
        self.cell_to_south_col = self.cell_col
        self.cell_index_to_south = get_index_from_rc(self.cell_to_south_row,self.cell_to_south_col)

        # Cell to the East
        self.cell_to_east_row = self.cell_row
        self.cell_to_east_col = self.cell_col + 1 if self.cell_col + 1 <= 15 else 15
        self.cell_index_to_east = get_index_from_rc(self.cell_to_east_row,self.cell_to_east_col)

        # Cell to the West
        self.cell_to_west_row = self.cell_row
        self.cell_to_west_col = self.cell_col - 1 if self.cell_col - 1 >= 0 else 0
        self.cell_index_to_west = get_index_from_rc(self.cell_to_west_row,self.cell_to_west_col)
        
        # Sets the (x,y) center of the cell relative the the maze (in mm)
        self.center_x = ((90) + (self.cell_col*180))
        self.center_y = (((180*size) - 90) - (self.cell_row*180))
        
        # Creates a list of wall object
        # TODO : fix so that no walls are dealt with
        self.walls = []
        for wall_indexer in range(4):
            w = wall(self.index, self.center_x, self.center_y, wall_indexer, self.wall_config[wall_indexer])
            self.walls.append(w)

class maze:
    # Construction of a Maze Object. Wall configuration can be known or unknown 
    def __init__(self, cell_codes=np.full(256, 'Unknown'), size = 16):
        self.maze = np.full((size, size), cell())
        self.fully_discovered = False
        indexer = 0
        for cell_code in cell_codes:
            r,c = divmod(indexer,size)
            cell_and_walls = cell(index=indexer,wall_code=cell_code)
            self.maze[r][c] = cell_and_walls
            indexer+=1
    
    # Updates cell and wall configuration in the Maze (default assumes robot is facing North if not provided)
    def set_cell_walls(self,cell_r,cell_c,sensor_readings,heading='North'):
        # Gets original indexer
        indexer = get_index_from_rc(cell_r,cell_c)
        # Gets wall config
        wall_config = sensor_readings_to_wall_config(sr=sensor_readings,heading=heading)
        print(wall_config)
        wall_code = list(wall_dict.keys())[list(wall_dict.values()).index(wall_config)]
        # Creates new cell object and updates the maze
        cell_and_walls = cell(index=indexer,wall_code=wall_code,discovered=True)
        self.maze[r][c] = cell_and_walls
        return 0

    # Updates fully discovered flag
    def is_fully_discovered(self):
        self.fully_discovered = True
        for maze_cell in self.maze:
            if not maze_cell.discovered:
                self.fully_discovered = False
        return self.fully_discovered
        
    
class robotPose:
    def __init__(cell_index, theta):
        
        self.cell_index             = cell_index
        self.cell_r, self.cell_c    = get_rc_from_index(self.cell_index)
        self.x, self.y              = get_xy_from_rc(self.cell_r, self.cell_y)
        self.theta                  = theta
        self.heading                = angle_to_heading(theta)
    def set_xy(self,x,y):
        self.x = x
        self.y = y
        self.cell_index = get_index_from_xy(self.x,self.y)
        self.cell_r, self.cell_c    = get_rc_from_index(self.cell_index)
    def set_cell_index(self, cell_index):
        self.cell_index = cell_index
        self.cell_r, self.cell_c    = get_rc_from_index(self.cell_index)
        self.x, self.y              = get_xy_from_rc(self.cell_r, self.cell_y)
    def set_theta(self, theta):
        self.theta = theta
        self.heading                = angle_to_heading(theta)


In [62]:
m = np.full(256, 'Unknown')
world_map=maze()

In [63]:
r,c = 15,15
sensor_readings = [.05,.05,1.5,.05]
heading = 'North'
cell_walls = world_map.maze[r][c].wall_config
for w in cell_walls:
    if w == '?':
        # print(world_map.maze[r][c].discovered)
        world_map.set_cell_walls(r,c,sensor_readings)
        break
cell_walls = world_map.maze[r][c].wall_config
if not world_map.maze[r][c].discovered:
    print("No")
else:
    print("Yes")
next_cell_r = world_map.maze[r][c].cell_index_to_west
print(next_cell_r)

[1, 1, 0, 1]
Yes
254


In [64]:
get_index_from_xy(2879,2879)

15

In [57]:
x = np.arange(0,256).reshape(16,16)
print(x)

[[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15]
 [ 16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31]
 [ 32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47]
 [ 48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63]
 [ 64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79]
 [ 80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95]
 [ 96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111]
 [112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127]
 [128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143]
 [144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159]
 [160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175]
 [176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191]
 [192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207]
 [208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223]
 [224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 